<a href="https://colab.research.google.com/github/dagousket/ML-course-VIB-2020/blob/master/Histone_Marks_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Let's go Deep Learning style!


In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import PIL
import tensorflow as tf

random_seed = 666
np.random.seed(random_seed)

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.metrics import log_loss, accuracy_score

### Import and format data

In [2]:
train = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data/data_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data/data_test.csv")

# Save labels in external variable
train_labels = train.pop("Label")
train_index_col = train.pop("GeneId")
test_index_col = test.pop("GeneId")

In [3]:
marks = list(set([c_name.split("_")[0] for c_name in train.columns]))

# Shape of array will be 100 bins * 5 histone marks

def pool_marks(mybins, marks) :
  df = pd.DataFrame()
  for mark in marks :
      df.reset_index()
      df[mark] = mybins[[mark + "_" + str(bin) for bin in range(0,100)]].to_numpy()
  df = df.to_numpy() / 200
  return df

def get_input(X, marks):
  res_out = X.apply(lambda y : pool_marks(y, marks), axis = 1)
  return pd.DataFrame(res_out)

X = [A for A in get_input(train, marks).iloc[:,0]]
X = np.array([np.array(dp) for dp in X])

T = [A for A in get_input(test, marks).iloc[:,0]]
T = np.array([np.array(dp) for dp in T])

In [4]:
print(X.shape)
print(T.shape)
X[:5]

(10436, 100, 5)
(5049, 100, 5)


array([[[0.005, 0.015, 0.005, 0.005, 0.01 ],
        [0.005, 0.005, 0.01 , 0.   , 0.025],
        [0.005, 0.015, 0.015, 0.025, 0.04 ],
        ...,
        [0.01 , 0.015, 0.04 , 0.03 , 0.08 ],
        [0.01 , 0.01 , 0.055, 0.025, 0.075],
        [0.01 , 0.01 , 0.05 , 0.01 , 0.06 ]],

       [[0.   , 0.   , 0.   , 0.005, 0.005],
        [0.01 , 0.01 , 0.   , 0.   , 0.01 ],
        [0.01 , 0.005, 0.   , 0.   , 0.005],
        ...,
        [0.   , 0.   , 0.005, 0.   , 0.005],
        [0.005, 0.   , 0.005, 0.   , 0.   ],
        [0.005, 0.   , 0.005, 0.   , 0.   ]],

       [[0.035, 0.005, 0.015, 0.005, 0.005],
        [0.02 , 0.01 , 0.01 , 0.005, 0.005],
        [0.01 , 0.   , 0.01 , 0.005, 0.02 ],
        ...,
        [0.01 , 0.005, 0.   , 0.005, 0.005],
        [0.005, 0.   , 0.005, 0.01 , 0.   ],
        [0.005, 0.   , 0.005, 0.005, 0.005]],

       [[0.02 , 0.015, 0.01 , 0.005, 0.02 ],
        [0.   , 0.015, 0.01 , 0.005, 0.005],
        [0.005, 0.005, 0.03 , 0.005, 0.005],
        ..

In [5]:
#one-hot encode target column
from keras.utils import to_categorical
train_labels_cat = to_categorical(train_labels)
train_labels_cat = np.array([Y for Y in train_labels_cat])
print(train_labels_cat.shape)
print(train_labels_cat[:5])
print(train_labels[:5])

(10436, 2)
[[0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]]
0    1
1    1
2    0
3    0
4    1
Name: Label, dtype: int64


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, train_labels_cat, test_size=0.15)

### Make a DL model

In [7]:
from sklearn.metrics import log_loss
import tensorflow as tf

def custom_loss_function(y_true, y_pred):
  custom_logloss = tf.compat.v1.losses.log_loss(y_true[:, 1], y_pred[:, 1])
  return custom_logloss

#y_true = y_train[:5]
#y_pred = predictions_train_prob[:5]
#cl = custom_loss_function(np.array(y_true),np.array(y_pred))
#print(cl)

In [18]:
from keras.layers.recurrent import LSTM

num_classes = 2

model = Sequential([
  layers.InputLayer(input_shape=(100,5)),
  layers.Conv1D(40, kernel_size=3, padding="valid", activation='relu', kernel_initializer='random_uniform', data_format ="channels_last"),
  layers.MaxPooling1D(pool_size=3, strides=1, padding='valid'),
  layers.Dropout(0.2),
  layers.TimeDistributed(layers.Dense(128, activation='relu')),
  layers.Bidirectional(LSTM(128, dropout=0.1, recurrent_dropout=0.1, return_sequences=True)),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(20, activation='relu'),
  layers.Dropout(0.4),
  layers.Dense(num_classes, activation='softmax')])



model.compile(optimizer='adam',
              loss = log_loss,
              #loss=custom_loss_function,
              metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 98, 40)            640       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 96, 40)            0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 96, 40)            0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 96, 128)           5248      
_________________________________________________________________
bidirectional_3 (Bidirection (None, 96, 256)           263168    
_________________________________________________________________
dropout_10 (Dropout)         (None, 96, 256)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 24576)            

In [19]:

history=model.fit(X_train, y_train, batch_size=100 , epochs=30 , verbose=1, validation_data=(X_test, y_test))


Epoch 1/30


NotImplementedError: ignored

In [10]:
from sklearn.metrics import log_loss, accuracy_score

predictions_train_prob = model.predict(X_train)
predictions_test_prob = model.predict(X_test)

print(predictions_train_prob[:5])
print(y_train[:5])
print(predictions_test_prob[:5])
print(y_test[:5])

[[0.20132858 0.21222237]
 [0.2610563  0.9309057 ]
 [0.3465991  0.83510494]
 [0.24061204 0.95789766]
 [0.07805056 0.02925246]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]]
[[0.32796597 0.8126326 ]
 [0.1719301  0.22842583]
 [0.22778289 0.9671692 ]
 [0.4003278  0.63338685]
 [0.3205513  0.9572353 ]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [11]:
# Log Loss from Tensor Flow
print(tf.compat.v1.losses.log_loss(y_train, predictions_train_prob))
print(tf.compat.v1.losses.log_loss(y_test, predictions_test_prob))

print(tf.compat.v1.losses.log_loss([y[1] for y in y_test], [p[1] for p in predictions_test_prob]))
print(tf.compat.v1.losses.log_loss([y[1] for y in y_test], [p[1] for p in predictions_test_prob]))

tf.Tensor(0.57755363, shape=(), dtype=float32)
tf.Tensor(0.626107, shape=(), dtype=float32)
tf.Tensor(0.39186803, shape=(), dtype=float32)
tf.Tensor(0.39186803, shape=(), dtype=float32)


In [12]:
# Log Loss from Keras
from sklearn.metrics import log_loss
print(log_loss([y[1] for y in y_train], [p[1] for p in predictions_train_prob]))
print(log_loss([y[1] for y in y_test], [p[1] for p in predictions_test_prob]))


0.32526182364640527
0.3918683130549366


In [13]:
# Predict Kaggle set
predictions_val_prob = model.predict(T)
predictions_val_prob[:5]

array([[0.24105927, 0.94305646],
       [0.20135136, 0.15971689],
       [0.18330292, 0.0492955 ],
       [0.2710751 , 0.8946958 ],
       [0.3179927 , 0.88545185]], dtype=float32)

In [38]:
H = [X[1] for X in predictions_val_prob]
H[:5]

[0.9747422, 0.2507878, 0.12457839, 0.9232302, 0.88581705]

In [40]:
predictions_df = pd.DataFrame({"GeneId":test_index_col,"Label":H})
print(predictions_df.head())
predictions_df.to_csv('submission_dl4.csv', index=False)

   GeneId     Label
0    5222  0.974742
1     891  0.250788
2    7219  0.124578
3    7225  0.923230
4    9432  0.885817
